# Importações

In [1]:
# Preparando ambiente (importando bibliotecas e downloads...)

!pip install nltk
import nltk
nltk.download('rslp')
nltk.download('stopwords')
nltk.download('punkt')
import re
import pandas as pd 
import numpy as np
import spacy
nlp = spacy.load('pt')

[nltk_data] Downloading package rslp to /home/karine/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /home/karine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/karine/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Tratamento Dados Originais


In [2]:

Logs_Original = open('logs/LogGeral_Original.txt', 'r', encoding='utf8')
Arquivo_Preprocessado = open('logs/LogGeral_PreProcessado.txt', 'w+')


In [3]:

agente = ''
for i in Logs_Original:
    novo_agente = 'nulo'
    #print('oi')
    if (i[-11:] == '06/01/2020\n') or (i[-11:] == '06/02/2020\n'):
        novo_agente = i[:-11]

    if novo_agente == 'nulo':
        Arquivo_Preprocessado.writelines(str(agente + '\t' + i))
    else: 
        if novo_agente == 'AgAt1 - Juliano Storch':
            agente = 'AgAT1'
        elif novo_agente == 'AgAt2':
            agente = 'AgAT2'
        elif novo_agente == 'AgCD1 - Fiscal Cidadela':
            agente = 'AgCD1|Fiscal CD'
        elif novo_agente == 'AgCD2 / PrefeitoCD - GustavoLima':
            agente = 'AgCD2|Prefeito CD'
        elif novo_agente == 'AgCD3 | VerCD':
            agente = 'AgCD3|Vereador CD'
        elif novo_agente == 'EmpMaq - PrefeitaATLANTIS':
            agente = 'EmpMaq|Prefeito AT'
        elif novo_agente == 'EmpSem / Fiscal - Bruno':
            agente = 'EmpSem|Fiscal AT'  


In [4]:

Logs_Original.close()
Arquivo_Preprocessado.close()


# Carregamento e tratamento dos léxicos e datasets necessários

In [5]:
dicionario_lexico = {}
sentilexpt = open('SentiLex-lem-PT02.txt','r', encoding='utf8')
ontolp = open('lexico_v3.0.txt','r', encoding='utf8')

In [6]:
for i in sentilexpt:
    pos_ponto = i.find('.')
    palavra = (i[:pos_ponto])
    pol_pos = i.find('POL')
    polaridade = (i[pol_pos+7:pol_pos+9]).replace(';','')
    dicionario_lexico[palavra] = polaridade

In [7]:
for i in ontolp:
    split_dic = i.split(',')
    palavra = split_dic[0]
    if palavra not in dicionario_lexico:
        polaridade = split_dic[2]
        dicionario_lexico[palavra] = polaridade        

In [8]:
degree_words = open('degree-words.txt','r', encoding='utf8')
degree_words_set = {}

In [9]:
for i in degree_words:
    split_dic = i.split(';')
    degree_words_set[split_dic[0]] = split_dic[1][:-1]

In [10]:
sentilexpt.close()
ontolp.close()
degree_words.close()

In [11]:
liwc = pd.read_csv("LIWC2007.txt",sep="\t",encoding="ISO-8859-1", names=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20'])

In [12]:
liwc_set = {}
def cria_dic_liwc(x):
    lista = []
    if 126 in list(x):
        liwc_set[x.name] = 'pos'
    elif 127 in list(x):
        liwc_set[x.name] = 'neg'

In [13]:
liwc.apply(lambda x: cria_dic_liwc(x), axis=1)

a          None
aba        None
abafa      None
abafad*    None
abafada    None
           ... 
último     None
último*    None
út*        None
úteis      None
útil       None
Length: 127161, dtype: object

In [14]:
logs = pd.read_csv("logs/LogGeral_PreProcessado.txt", sep="\t" , encoding="utf8", names=['Agente', 'Frase'])

# Taxa de sentimento por frase

In [19]:
df = pd.read_csv("dataset.csv", sep="," , encoding="utf8")
df = df['text_pt'][:10]
for frase in logs['Frase']:
    
    frase = nlp(frase)
    degree_words_head = []
    degree_words_aux = []
    score_frase = {'pos':0, 'neg':0}
    
    for token in frase:
        if(token.text in degree_words_set):
            degree_words_head.append(token.head.text)
            degree_words_aux.append(int(degree_words_set.get(token.text)))
            
    for token in frase:
        if(token.text in liwc_set):
            if token.head.text in degree_words_head:
                dic = liwc_set.get(token.text)
                neg = int(degree_words_aux[degree_words_head.index(token.head.text)])                   
                if dic == 'pos':
                    score_frase['pos'] = int(score_frase.get('pos')) + 1 + neg
                else:
                    score_frase['neg'] = int(score_frase.get('neg')) - 1 - neg

    #print(score_frase)